In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import random
import os
import dgl
from matplotlib import pylab

import networkx as nx

import sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import copy
from scipy import stats
from model import PopNet
from utils import eval, MAPE

Using backend: pytorch


In [2]:
def seed_torch(seed=6):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_torch()

# Data preparation

In [3]:
x_train_real = pickle.load(open('./data/x_train_real','rb'))
x_train_update = pickle.load(open('./data/x_train_update','rb'))
y_train_real = pickle.load(open('./data/y_train_real','rb'))
y_train_update = pickle.load(open('./data/y_train_update','rb'))


x_val_real = pickle.load(open('./data/x_val_real','rb'))
x_val_update = pickle.load(open('./data/x_val_update','rb'))
y_val = pickle.load(open('./data/y_val','rb'))


x_test_real = pickle.load(open('./data/x_test_real','rb'))
x_test_update = pickle.load(open('./data/x_test_update','rb'))
y_test = pickle.load(open('./data/y_test','rb'))


normalizer = pickle.load(open('./data/normalizer','rb'))
g = pickle.load(open('./data/g','rb'))
x_static = pickle.load(open('./data/x_static','rb'))

In [4]:
print('X train real shape {0}'.format(x_train_real.shape))
print('X train update shape {0}'.format(x_train_update.shape))
print('Y train shape {0}'.format(y_train_real.shape))
print('Y train update shape {0}'.format(y_train_update.shape))

print()
print('X val real shape {0}'.format(x_val_real.shape))
print('X val update shape {0}'.format(x_val_update.shape))
print('Y val shape {0}'.format(y_val.shape))

print()
print('X test real shape {0}'.format(x_test_real.shape))
print('X test update shape {0}'.format(x_test_update.shape))
print('Y test shape {0}'.format(y_test.shape))


X train real shape (1015, 63, 22)
X train update shape (1015, 63, 22)
Y train shape (1015, 63, 1)
Y train update shape (1015, 51)

X val real shape (1015, 19, 22)
X val update shape (1015, 19, 22)
Y val shape (1015, 19, 1)

X test real shape (1015, 19, 22)
X test update shape (1015, 19, 22)
Y test shape (1015, 19, 1)


# Define model

In [5]:
in_dim = 22
static_dim = 4
out_dim = 1
gat_dim1 = 32
gat_dim2 = 22
satt_dim = 32
sie_dim = 32
tatt_dim = 64
gru_dim = 256
fc_dim = 256
num_heads = 1

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
g = g.to(device)
model = PopNet(g, in_dim, static_dim, out_dim, gat_dim1, gat_dim2, satt_dim, sie_dim, tatt_dim, gru_dim, fc_dim, num_heads, device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

In [6]:
print(model)

PopNet(
  (nn_r_gat1): MultiHeadGATLayer(
    (heads): ModuleList(
      (0): GATLayer(
        (fc): Linear(in_features=22, out_features=32, bias=True)
        (attn_fc): Linear(in_features=64, out_features=1, bias=True)
      )
    )
  )
  (nn_u_gat1): MultiHeadGATLayer(
    (heads): ModuleList(
      (0): GATLayer(
        (fc): Linear(in_features=22, out_features=32, bias=True)
        (attn_fc): Linear(in_features=64, out_features=1, bias=True)
      )
    )
  )
  (nn_input): Linear(in_features=22, out_features=22, bias=True)
  (nn_satt): CrossGraphGAT(
    (fc_static): Linear(in_features=4, out_features=32, bias=True)
    (fc_real): Linear(in_features=64, out_features=32, bias=True)
    (fc_update): Linear(in_features=64, out_features=32, bias=True)
    (fc_attn): Linear(in_features=32, out_features=1, bias=True)
  )
  (nn_r_gru): GRUCell(86, 256)
  (nn_u_gru): GRUCell(54, 256)
  (nn_u_conv1): CausalConv1d(22, 16, kernel_size=(3,), stride=(1,), padding=(2,))
  (nn_u_conv2): Causa

# Testing PopNet

In [7]:
x_train_real = torch.tensor(x_train_real).to(device)
x_train_update = torch.tensor(x_train_update).to(device)
y_train_real = torch.tensor(y_train_real).to(device)
y_train_update = torch.tensor(y_train_update).to(device)

x_val_real = torch.tensor(x_val_real).to(device)
x_val_update = torch.tensor(x_val_update).to(device)
y_val = torch.tensor(y_val).to(device)

x_test_real = torch.tensor(x_test_real).to(device)
x_test_update = torch.tensor(x_test_update).to(device)
y_test = torch.tensor(y_test).to(device)

x_static = torch.tensor(x_static).to(device)

In [8]:
file_name = './save/PopNet'

checkpoint = torch.load(file_name)
model.load_state_dict(checkpoint['state'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

PopNet(
  (nn_r_gat1): MultiHeadGATLayer(
    (heads): ModuleList(
      (0): GATLayer(
        (fc): Linear(in_features=22, out_features=32, bias=True)
        (attn_fc): Linear(in_features=64, out_features=1, bias=True)
      )
    )
  )
  (nn_u_gat1): MultiHeadGATLayer(
    (heads): ModuleList(
      (0): GATLayer(
        (fc): Linear(in_features=22, out_features=32, bias=True)
        (attn_fc): Linear(in_features=64, out_features=1, bias=True)
      )
    )
  )
  (nn_input): Linear(in_features=22, out_features=22, bias=True)
  (nn_satt): CrossGraphGAT(
    (fc_static): Linear(in_features=4, out_features=32, bias=True)
    (fc_real): Linear(in_features=64, out_features=32, bias=True)
    (fc_update): Linear(in_features=64, out_features=32, bias=True)
    (fc_attn): Linear(in_features=32, out_features=1, bias=True)
  )
  (nn_r_gru): GRUCell(86, 256)
  (nn_u_gru): GRUCell(54, 256)
  (nn_u_conv1): CausalConv1d(22, 16, kernel_size=(3,), stride=(1,), padding=(2,))
  (nn_u_conv2): Causa

In [9]:
model.eval()

#Get previous hidden state
prev_input1 = torch.cat((x_train_real, x_val_real), dim=1)
prev_input2 = torch.cat((x_train_update, x_val_update), dim=1)
_, _, test_hidden1, test_hidden2, kl = model(prev_input1, prev_input2, x_static, None, None, None)

#Set normailize to original value range
norm = True
actual_pred, actual_true = eval(model, x_test_real, x_test_update, y_test, x_static, test_hidden1, test_hidden2, None, normalizer, norm)


In [10]:
mse = []
for i in range(len(actual_pred)):
    mse.append(mean_squared_error(actual_true[i],actual_pred[i], squared='False'))
print('RMSE: %.2f'%np.mean(mse))
mae = []
for i in range(len(actual_pred)):
    mae.append(mean_absolute_error(actual_true[i],actual_pred[i]))
print('MAE: %.2f'%np.mean(mae))
mape = []
for i in range(len(actual_pred)):
    mape.append(MAPE(actual_true[i],actual_pred[i]))
print('MAPE: %.2f'%np.mean(mape))

RMSE: 177610.69
MAE: 97.94
MAPE: 33.00


# Training PopNet from scratch

In [ ]:
all_loss = []

file_name = './save/PopNet_new'
for epoch in range(200):
    model.train()
    
    y_pred_r, y_pred_u, hidden1, hidden2, kl = model(x_train_real, x_train_update, x_static, None, None, None) 
    loss = criterion(y_pred_r, y_train_real) + criterion(y_pred_u[:, 12:], y_train_update.unsqueeze(-1)) + kl
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    all_loss.append(loss.item())
    state = {
        'state': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    torch.save(state, file_name)
    
    print('Epoch %d, Loss %.2f, KL loss %.2f'%(epoch, all_loss[-1], kl))